# Sesión 5 B

## La distribución previa como regularizador

> **Objetivos:**
> - Comprender el principio de estimación Máximum A-Posteriori (MAP).
> - Entender el efecto de la distribución previa como regularizador.

### I. Introducción

En este cuaderno de trabajo exploraremos un enfoque _intermedio_ entre la perspectiva bayesiana y la frecuentista.

#### Recordemos primero la visión frecuentista

En el enfoque frecuentista aparece el concepto de **estimadores puntuales**. 

```{admonition} Estimadores puntuales
:class: note

Un estimador puntual es una función de los datos observados que proporciona una _única estimación_ del valor de un parámetro desconocido.
```

Desde esta perspectiva:

* Tenemos una situación con incertidumbre.

* Para describirla, planteamos un modelo parametrizado por un conjunto de parámetros desconocidos.

* El objetivo es encontrar el valor de esos parámetros que mejor se ajuste a los datos disponibles.

* En notación matemática, podríamos escribir el modelo como:

$$
M(\theta) = p(\theta|D)
$$

#### Ideas clave en la visión frecuentista

* Los parámetros *existen* y son *valores fijos*, aunque no los conozcamos.

* Lo que queremos es encontrar esos valores.

* La incertidumbre *no está en los parámetros*, sino en los datos.

Como consecuencia, aparece la función de verosimilitud: 

$$L(\theta|D)$$

que mide qué tan probable es observar los datos bajo diferentes valores de $\theta$.

#### La perspectiva bayesiana

Veamos ahora el mismo problema desde la estadística bayesiana.

Partimos de la mismo situación:

* Tenemos incertidumbre

* Contamos con un modelo parametrizado

* Y disponemos de un conjunto de datos observados

La diferencia clave está en cómo entendemos la _fuente de la incertidumbre:_

* Para los frecuentistas, los datos son aleatorios y los parámetros son fijos (aunque desconocidos).

* Para los bayesianos, sucede al revés:
    * Los datos ya están fijos (no hay incertidumbre en ellos una vez observados).

    * La incertidumbre está en los parámetros, porque no conocemos sus valores. 

```{admonition} En otras palabras
:class: tip

Como _bayesianos_ no nos preguntamos únicamente _"¿cuál es el valor de $\theta$ que mejor explica los datos?"_, sino _"¿cuál es la probabilidad de cada posible valor de $\theta$, dado lo que ya observamos?"_.

Matemáticamente, expresamos esta idea con la **distribución posterior**:

$$
p(\theta | D)
$$

donde $D$ son los datos (fijos) y $\theta$ representa los parámetros (inciertos). 

#### Un punto intermedio: el estimador MAP

Hemos visto dos perspectivas:

* Frecuentista: los parámetros son **valores fijos** (desconocidos) y la incertidumbre está en los **datos**.

* Bayesiana: los parámetros tienen una **distribución de probabilidad** y la incertidumbre está en los **parámetros**, mientras que los datos observados son **valores fijos**.

Entre estos dos enfoques surge una visión _intermedia_, que da lugar a otros tipo de estimador: el **estimador de máximo a posteriori (MAP)**.


```{admonition} ¿qué significa el MAP?
:class: note

- Desde Bayes sabemos que la *distribución posterior* es proporcional a la *verosimilitud* multiplicada por la *distribución previa*:  

$$
p(\theta \mid D) \propto L(\theta \mid D) \, p(\theta)
$$

- En lugar de trabajar con toda la distribución posterior (como en Bayes puro), buscamos *el valor de $\theta$ que maximiza esta posterior*.  

Es decir, definimos el estimador MAP como:  

$$
\hat{\theta}_{MAP} = \arg\max_\theta \; \big[ \; L(\theta \mid D)\, p(\theta) \;\big]
$$
```

#### Conexión con la regularización

Aquí aparece algo muy interesante que veremos a continuación, pero antes introduzcamos la distribución normal multivariable, puesto que nos será útil para obtener la función de densidad de probabilidad normal multivariada:

$$
\mathcal{N}(x|\mu, \Sigma) = \frac{1}{\sqrt{\det(2 \pi \Sigma)}} \exp \left\{-\frac{1}{2}(x - \mu)^T \Sigma^{-1} (x - \mu)\right\},
$$

con parámetros $\mu \in \mathbb{R}^d$: vector de medias de la V.A. $X$ y $\Sigma \in \mathbb{R}^{d \times d}$: matriz de covarianzas de la VA X (simétrica y definida positiva).

In [1]:
from scipy import stats
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [5]:
X = stats.multivariate_normal(
    mean=[0, 0],
    cov=[
        [1, 0],
        [0, 1]
    ]
)
Y = stats.multivariate_normal(
    mean=[1, 1],
    cov=[
        [1, -0.8],
        [-0.8, 1]
    ]
)

```{admonition} Intuición
:class: tip

Una normal multivariada puede imaginarse como una campana en varias dimensiones.

* La media marca el centro de la campana.

* La covarianza define la forma:

  * sin correlación → contornos circulares,

  * con correlación → contornos elípticos e inclinados.
```

![](../images/sesion5_multi1.png)

**Figura 1.** Distribución normal bivariada con media en $(0,0)$ y matriz de covarianza diagonal. Los contornos circulares reflejan independencia y dispersión igual en todas las direcciones.

![](../images/sesion5_multi2.png)

**Figura 2.** Distribución normal bivariada con media en $(1,1)$ y covarianza no diagonal. Los contornos elípticos inclinados muestran la correlación negativa entre las variables.

```{admonition} Matriz de covarianzas
:class: tip

La matriz de covarianza describe cómo varían las variables entre sí:

* Diagonal → varianzas de cada variable (qué tan dispersas están).

* Fuera de la diagonal → covarianzas (si aumentan o disminuyen juntas).

* Valores positivos → relación directa,

* Valores negativos → relación inversa,

* Ceros → independencia (sin relación lineal).

En una normal multivariada, esta matriz es la que da forma y orientación a los contornos (círculos o elipses).

### II. Retomemos ajuste de curvas

Con lo anterior dicho, volvamos a hablar del tema de *ajuste de curvas* con una perspectiva probabilística. 

Partimos de nuevo modelando la relación entre nuestra variable de salida $y$ y las variables de entrada $x$ como un **modelo lineal** con incertidumbre, la cual suponemos normal:

$$
y = \phi(x)^T w + \epsilon,
$$

con $\epsilon \sim \mathcal{N}(0, \beta^{-1})$.

Equivalentemente, podemos decir que:

$$
y \sim \mathcal{N}(\phi(x)^T w, \beta^{-1}),
$$

con esta suposición podemos determinar que la verosimilitud de un datos, sigue una distribución normal con:

$$
p(y | x, w) = \mathcal{N}(y | \phi(x)^T w, \beta^{-1}).
$$

Ya sabemos a donde nos conduce este modelo para la función de verosimilitud. 

Alternativamente, encontremos una expresión para la distribución posterior de los parámetros, usando la regla de Bayes:

\begin{align}
p(w | y, X) & = \frac{p(y, w | X)}{p(y | X)} \qquad \text{(Definición de probabilidad condicional)} \\
            & = \frac{p(y | X, w) p(w | X)}{p(y | X)} \qquad \text{(Bayes)}.
\end{align}

En este contexto, tenemos que:

- $p(w | y, X)$ es la distribución posterior de los parámetros dados los datos.

- $p(y | X, w) = \mathcal{L}(w) = \prod_{i=1}^{N} \mathcal{N}(y_i | \phi(x_i)^T w, \beta^{-1})$ es la función de verosimilitud.

- $p(y | X)$ es la distribución de evidencia.

- $p(w | X)$ es la distribución previa. Para este punto, una suposición que hace bastante sentido es que conocer únicamente las variables de entrada $X$ no nos dice absolutamente nada acerca de los parámetros $w$. Es decir, **son independientes**. Por tanto,

  $$
  p(w | X) = p(w).
  $$ 

#### Maximizar la distribución posterior (MAP)

Aquí convergen los dos mundos, es decir, razonamos acerca de la **distribución posterior** de los parámetros con Bayes, pero **tomaremos un valor puntual del parámetro más probable**. 

Maximizamos la distribución posterior:

\begin{align}
\hat{w}_{MAP} & = \arg \max_{w} p(w | y, X) \\
              & = \arg \max_{w} \frac{p(y | X, w) p(w | X)}{p(y | X)} \\
              & = \arg \max_{w} p(y | X, w) p(w | X) \qquad \text{(La distribución de evidencia no depende de $w$)} \\
              & = \arg \max_{w} \log p(y | X, w) + \log p(w) \qquad \text{(El logaritmo es una función creciente y cóncava)} \\
              & = \arg \max_{w} l(w) + \log p(w)
\end{align}

El término de log-verosimilitud ya lo conocemos:

$$
l(w) = \frac{N}{2}\log\beta - \frac{N}{2}\log(2 \pi) - \frac{\beta}{2} \left|\left|y - \Phi w\right|\right|^2,
$$

La pregunta es, **¿Qué es la distribución previa $p(w)$?**

- Este término nos permite expresar todo nuestro conocimiento previo acerca de los parámetros $w$ de una manera probabilística, es decir, a través de una distribución de probabilidad.

#### Previa normal

Una elección común es:
  
\begin{align}
p(w) & = \mathcal{N}(w | 0, \alpha^{-1} I) \\
   & = \frac{1}{\sqrt{\det(2 \pi \alpha^{-1} I)}} \exp \left\{-\frac{\alpha}{2} w^T w\right\} \\
   & = \frac{\alpha^{d / 2}}{(2 \pi)^{d / 2}} \exp \left\{-\frac{\alpha}{2} \left|\left|w\right|\right|^2\right\}. \\
\end{align}

En este caso, podemos usar el parámetro $\alpha$ para expresar cuanta certeza tenemos de que $w$ está cercano a cero.

De lo anterior, observamos que:

$$
\log p(w) 
= \textcolor{blue}{\log \frac{\alpha^{d / 2}}{(2 \pi)^{d / 2}}} 
-  \textcolor{blue}{\frac{\alpha}{2} \left|\left|w\right|\right|^2}
$$

Reemplazando en la expresión anterior y descartando todos los sumandos que no dependen de $w$:

\begin{align}
\hat{w}_{MAP} & = \arg \max_{w} l(w) + \log p(w) \\
              & = \arg \max_{w} - \frac{\beta}{2} \left|\left|y - \Phi w\right|\right|^2 - \frac{\alpha}{2} \left|\left|w\right|\right|^2 \\
              & = \arg \min_{w} \left|\left|y - \Phi w\right|\right|^2 + \underbrace{\frac{\alpha}{\beta}}_{\lambda} \left|\left|w\right|\right|^2.
\end{align}

Entonces, observamos que la estimación de parámetros por MAP, usando una previa Gaussiana nos conduce a nuestra intuición detrás de **mínimos cuadrados regularizados con norma-$2$ (Ridge)**.

#### Previa de Laplace

Otra previa común en este caso, es modelar la incertidumbre en los parámetros como una distribución de Laplace:

$$
p(w_i) = \frac{1}{2 b} \exp\left(-\frac{|w_i|}{b}\right),
$$

con lo cual, suponiendo que los parámetros son independientes:

\begin{align}
p(w) & = \prod_{i=1}^{d} p(w_i) \\
     & = \frac{1}{(2 b)^{d}} \exp\left(-\sum_{i=1}^{d}\frac{|w_i|}{b}\right) \\
     & = \frac{1}{(2 b)^{d}} \exp\left(-\frac{1}{b} \sum_{i=1}^{d}|w_i|\right). \\
\end{align}

En este caso, observamos que:

$$
\log p(w) 
= \textcolor{blue}{\log \frac{1}{(2 b)^{d}}} 
- \textcolor{blue}{\frac{1}{b} \sum_{i=1}^{d}|w_i|}
$$

Con lo cual

\begin{align}
\hat{w}_{MAP} & = \arg \min_{w} \left|\left|y - \Phi w\right|\right|^2 + \underbrace{\frac{1}{\beta b}}_{\lambda}  \sum_{i=1}^{d}|w_i|.
\end{align}


Entonces, observamos que la estimación de parámetros por MAP, usando una previa de Laplace nos conduce a nuestra intuición detrás de **mínimos cuadrados regularizados con norma-$1$ (Lasso)**.

### III. Conclusiones

1. Desde la **perspectiva probabilística**, podemos reinterpretar el problema de ajuste de curvas y recuperar resultados clásicos de forma intuitiva.  

2. Los **estimadores de máxima verosimilitud (MLE)** funcionan bien con muchos datos, pero tienden a sufrir de _overfitting_ cuando la muestra es pequeña.  

3. Los **estimadores de máxima a posteriori (MAP)** incorporan información previa sobre los parámetros mediante una distribución _prior_, lo que actúa como un mecanismo natural de **regularización**.